# Quantlib技术分析实战演示

本笔记本展示如何使用quantlib技术指标模块对真实股票数据进行技术分析。

## 目标
1. 获取真实股票数据
2. 使用quantlib.technical进行全面技术分析
3. 生成交易信号和投资建议
4. 可视化分析结果

In [17]:
# 导入必要的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

print("库导入成功")

库导入成功


In [3]:
# 导入quantlib技术分析模块
import sys
import os
sys.path.append('.')

from quantlib.technical import TechnicalAnalyzer
from quantlib.technical.trend import TrendIndicators
from quantlib.technical.oscillator import OscillatorIndicators
from quantlib.technical.volume import VolumeIndicators

print("quantlib技术分析模块导入成功")

quantlib技术分析模块导入成功


## 1. 获取股票数据

我们将分析几只热门股票的技术指标

In [4]:
# 定义要分析的股票
stocks = {
    'AAPL': 'Apple Inc.',
    'TSLA': 'Tesla Inc.',
    'GOOGL': 'Alphabet Inc.',
    'MSFT': 'Microsoft Corp.'
}

# 设置数据获取参数
period = '6mo'  # 6个月数据
interval = '1d'  # 日线数据

print(f"准备获取 {len(stocks)} 只股票的数据...")
print(f"时间周期: {period}, 数据间隔: {interval}")

准备获取 4 只股票的数据...
时间周期: 6mo, 数据间隔: 1d


In [5]:
def get_stock_data(symbol, period='6mo'):
    """获取股票数据"""
    try:
        ticker = yf.Ticker(symbol)
        data = ticker.history(period=period)
        
        # 标准化列名为小写
        data.columns = [col.lower() for col in data.columns]
        data = data.reset_index()
        
        # 确保日期列名正确
        if 'date' not in data.columns and 'Date' in data.columns:
            data['date'] = data['Date']
            data = data.drop('Date', axis=1)
        elif data.index.name == 'Date':
            data['date'] = data.index
            data = data.reset_index(drop=True)
        
        print(f"✓ {symbol}: 获取 {len(data)} 条记录 ({data['date'].iloc[0].strftime('%Y-%m-%d')} 至 {data['date'].iloc[-1].strftime('%Y-%m-%d')})")
        return data
        
    except Exception as e:
        print(f"× {symbol}: 获取数据失败 - {e}")
        return None

# 获取所有股票数据
stock_data = {}
for symbol, name in stocks.items():
    data = get_stock_data(symbol, period)
    if data is not None:
        stock_data[symbol] = data

print(f"\n成功获取 {len(stock_data)} 只股票的数据")

✓ AAPL: 获取 127 条记录 (2025-02-28 至 2025-08-29)
✓ TSLA: 获取 127 条记录 (2025-02-28 至 2025-08-29)
✓ GOOGL: 获取 127 条记录 (2025-02-28 至 2025-08-29)
✓ MSFT: 获取 127 条记录 (2025-02-28 至 2025-08-29)

成功获取 4 只股票的数据


## 2. 单只股票详细技术分析

以Apple (AAPL)为例进行详细的技术分析

In [6]:
# 选择AAPL进行详细分析
symbol = 'AAPL'
if symbol in stock_data:
    aapl_data = stock_data[symbol]
    print(f"分析股票: {symbol} - {stocks[symbol]}")
    print(f"数据概览:")
    print(f"- 记录数: {len(aapl_data)}")
    print(f"- 时间范围: {aapl_data['date'].iloc[0].strftime('%Y-%m-%d')} 至 {aapl_data['date'].iloc[-1].strftime('%Y-%m-%d')}")
    print(f"- 当前价格: ${aapl_data['close'].iloc[-1]:.2f}")
    print(f"- 期间涨跌: {((aapl_data['close'].iloc[-1] / aapl_data['close'].iloc[0] - 1) * 100):.2f}%")
    
    print("\n数据样本:")
    print(aapl_data[['date', 'open', 'high', 'low', 'close', 'volume']].head())
else:
    print(f"未能获取{symbol}数据")

分析股票: AAPL - Apple Inc.
数据概览:
- 记录数: 127
- 时间范围: 2025-02-28 至 2025-08-29
- 当前价格: $232.14
- 期间涨跌: -3.78%

数据样本:
                       date        open        high         low       close  \
0 2025-02-28 00:00:00-05:00  236.371422  241.498871  229.637904  241.249481   
1 2025-03-03 00:00:00-05:00  241.199590  243.434125  235.533466  237.448776   
2 2025-03-04 00:00:00-05:00  237.129580  239.483818  234.106965  235.353912   
3 2025-03-05 00:00:00-05:00  234.845157  235.972403  228.670269  235.164383   
4 2025-03-06 00:00:00-05:00  233.867560  237.279208  232.590687  234.755386   

     volume  
0  56833400  
1  47184000  
2  53798100  
3  47227600  
4  45170400  


In [7]:
# 创建技术分析器
analyzer = TechnicalAnalyzer(aapl_data)

# 计算所有技术指标
print("计算技术指标...")
analyzer.calculate_all_indicators()
print(f"完成! 计算了 {len(analyzer.indicators)} 个技术指标")

# 生成交易信号
print("\n生成交易信号...")
analyzer.generate_all_signals()
print(f"完成! 生成了 {len(analyzer.signals)} 个交易信号")

计算技术指标...
计算技术指标...
技术指标计算完成
完成! 计算了 16 个技术指标

生成交易信号...
生成交易信号...
交易信号生成完成
完成! 生成了 16 个交易信号


### 2.1 趋势指标分析

In [8]:
print("=== 趋势指标分析 ===")
print()

# 移动平均线
ma_results = analyzer.indicators['ma'].results
current_price = aapl_data['close'].iloc[-1]

print("1. 移动平均线 (Moving Averages):")
print(f"   当前价格: ${current_price:.2f}")
for key, ma_series in ma_results.items():
    if not ma_series.dropna().empty:
        ma_value = ma_series.iloc[-1]
        trend = "上方" if current_price > ma_value else "下方"
        deviation = ((current_price - ma_value) / ma_value) * 100
        print(f"   {key}: ${ma_value:.2f} (价格在均线{trend}, 偏离{deviation:+.1f}%)")

# MACD
macd_results = analyzer.indicators['macd'].results
print(f"\n2. MACD指标:")
print(f"   MACD线: {macd_results['MACD'].iloc[-1]:.4f}")
print(f"   信号线: {macd_results['Signal'].iloc[-1]:.4f}")
print(f"   柱状图: {macd_results['Histogram'].iloc[-1]:.4f}")

macd_trend = "金叉" if macd_results['MACD'].iloc[-1] > macd_results['Signal'].iloc[-1] else "死叉"
macd_position = "零轴上方" if macd_results['MACD'].iloc[-1] > 0 else "零轴下方"
print(f"   状态: MACD在{macd_position}, 相对信号线{macd_trend}")

# 布林带
bb_results = analyzer.indicators['bb'].results
print(f"\n3. 布林带 (Bollinger Bands):")
print(f"   上轨: ${bb_results['Upper_Band'].iloc[-1]:.2f}")
print(f"   中轨: ${bb_results['Middle_Band'].iloc[-1]:.2f}")
print(f"   下轨: ${bb_results['Lower_Band'].iloc[-1]:.2f}")
print(f"   价格位置: {bb_results['BB_Position'].iloc[-1]:.1f}%")

bb_position = bb_results['BB_Position'].iloc[-1]
if bb_position > 80:
    bb_signal = "超买区域，接近上轨"
elif bb_position < 20:
    bb_signal = "超卖区域，接近下轨"
else:
    bb_signal = "正常区域，在中轨附近"
print(f"   信号: {bb_signal}")

=== 趋势指标分析 ===

1. 移动平均线 (Moving Averages):
   当前价格: $232.14
   SMA_5: $230.33 (价格在均线上方, 偏离+0.8%)
   EMA_5: $230.79 (价格在均线上方, 偏离+0.6%)
   SMA_10: $229.18 (价格在均线上方, 偏离+1.3%)
   EMA_10: $229.28 (价格在均线上方, 偏离+1.2%)
   SMA_20: $225.70 (价格在均线上方, 偏离+2.9%)
   EMA_20: $225.74 (价格在均线上方, 偏离+2.8%)
   SMA_50: $215.38 (价格在均线上方, 偏离+7.8%)
   EMA_50: $218.20 (价格在均线上方, 偏离+6.4%)
   EMA_200: $211.89 (价格在均线上方, 偏离+9.6%)

2. MACD指标:
   MACD线: 4.8499
   信号线: 4.9529
   柱状图: -0.1030
   状态: MACD在零轴上方, 相对信号线死叉

3. 布林带 (Bollinger Bands):
   上轨: $244.00
   中轨: $225.70
   下轨: $207.40
   价格位置: 67.6%
   信号: 正常区域，在中轨附近


### 2.2 震荡指标分析

In [9]:
print("=== 震荡指标分析 ===")
print()

# RSI
rsi_value = analyzer.indicators['rsi'].results['RSI'].iloc[-1]
print(f"1. RSI相对强弱指标: {rsi_value:.1f}")
if rsi_value > 70:
    rsi_signal = "超买，考虑卖出"
elif rsi_value < 30:
    rsi_signal = "超卖，考虑买入"
else:
    rsi_signal = "正常区域，保持观望"
print(f"   信号: {rsi_signal}")

# KDJ
kdj_results = analyzer.indicators['kdj'].results
k_value = kdj_results['K'].iloc[-1]
d_value = kdj_results['D'].iloc[-1]
j_value = kdj_results['J'].iloc[-1]

print(f"\n2. KDJ随机指标:")
print(f"   K值: {k_value:.1f}")
print(f"   D值: {d_value:.1f}")
print(f"   J值: {j_value:.1f}")

kdj_trend = "金叉" if k_value > d_value else "死叉"
if k_value > 80 and d_value > 80:
    kdj_signal = "高位死叉，卖出信号"
elif k_value < 20 and d_value < 20:
    kdj_signal = "低位金叉，买入信号"
else:
    kdj_signal = f"K线相对D线{kdj_trend}，观察趋势"
print(f"   信号: {kdj_signal}")

# Williams %R
williams_value = analyzer.indicators['williams'].results['Williams_R'].iloc[-1]
print(f"\n3. Williams %R: {williams_value:.1f}")
if williams_value > -20:
    williams_signal = "超买，考虑卖出"
elif williams_value < -80:
    williams_signal = "超卖，考虑买入"
else:
    williams_signal = "正常区域"
print(f"   信号: {williams_signal}")

=== 震荡指标分析 ===

1. RSI相对强弱指标: 60.4
   信号: 正常区域，保持观望

2. KDJ随机指标:
   K值: 71.4
   D值: 60.3
   J值: 93.5
   信号: K线相对D线金叉，观察趋势

3. Williams %R: -26.3
   信号: 正常区域


### 2.3 成交量指标分析

In [10]:
print("=== 成交量指标分析 ===")
print()

# OBV
obv_current = analyzer.indicators['obv'].results['OBV'].iloc[-1]
obv_ma = analyzer.indicators['obv'].results['OBV_MA'].iloc[-1]
print(f"1. OBV能量潮: {obv_current:,.0f}")
print(f"   OBV均线: {obv_ma:,.0f}")
obv_trend = "上升" if obv_current > obv_ma else "下降"
print(f"   趋势: OBV相对均线{obv_trend}")

# VWAP
vwap_current = analyzer.indicators['vwap'].results['VWAP'].iloc[-1]
print(f"\n2. VWAP成交量加权均价: ${vwap_current:.2f}")
vwap_signal = "上方" if current_price > vwap_current else "下方"
print(f"   当前价格在VWAP{vwap_signal}")

# CMF蔡金资金流量
cmf_value = analyzer.indicators['cmf'].results['CMF'].iloc[-1]
print(f"\n3. CMF蔡金资金流量: {cmf_value:.3f}")
if cmf_value > 0.1:
    cmf_signal = "资金净流入"
elif cmf_value < -0.1:
    cmf_signal = "资金净流出"
else:
    cmf_signal = "资金流动平衡"
print(f"   信号: {cmf_signal}")

=== 成交量指标分析 ===

1. OBV能量潮: 176,496,100
   OBV均线: 163,937,710
   趋势: OBV相对均线上升

2. VWAP成交量加权均价: $208.64
   当前价格在VWAP上方

3. CMF蔡金资金流量: 0.164
   信号: 资金净流入


### 2.4 综合分析与交易建议

In [11]:
# 获取综合信号
signal, strength, analysis = analyzer.get_consensus_signal()

# 信号解读
signal_meaning = {
    2: "强烈看涨",
    1: "看涨", 
    0: "中性",
    -1: "看跌",
    -2: "强烈看跌"
}

print("=== 综合技术分析结果 ===")
print()
print(f"股票代码: {symbol}")
print(f"当前价格: ${current_price:.2f}")
print(f"综合信号: {signal_meaning[signal]} (信号值: {signal})")
print(f"信号强度: {strength:.2f} (0-1, 越高越可靠)")
print(f"信号一致性: {analysis['signal_consistency']:.2f}")
print()
print("信号分解:")
print(f"  趋势信号: {analysis['trend_signal']:.2f}")
print(f"  震荡信号: {analysis['oscillator_signal']:.2f}")
print(f"  成交量信号: {analysis['volume_signal']:.2f}")
print()
print("指标统计:")
print(f"  看涨指标: {analysis['bullish_count']} 个")
print(f"  看跌指标: {analysis['bearish_count']} 个")
print(f"  中性指标: {analysis['neutral_count']} 个")
print(f"  总指标数: {analysis['total_indicators']} 个")

# 交易建议
print("\n=== 交易建议 ===")
if signal >= 1:
    print(f"建议: 买入 / 持有")
    print(f"理由: {analysis['bullish_count']} 个看涨指标占主导")
elif signal <= -1:
    print(f"建议: 卖出 / 观望")
    print(f"理由: {analysis['bearish_count']} 个看跌指标占主导")
else:
    print(f"建议: 保持观望")
    print(f"理由: 多空信号相对均衡")

# 风险提示
risk_level = "低" if strength > 0.7 else "中" if strength > 0.4 else "高"
print(f"\n信号可靠性: {risk_level} (基于信号强度 {strength:.2f})")
if strength < 0.5:
    print("⚠️ 注意: 信号强度较低，建议结合其他分析方法")

=== 综合技术分析结果 ===

股票代码: AAPL
当前价格: $232.14
综合信号: 中性 (信号值: 0)
信号强度: 0.03 (0-1, 越高越可靠)
信号一致性: 0.64

信号分解:
  趋势信号: -0.20
  震荡信号: -0.17
  成交量信号: 1.00

指标统计:
  看涨指标: 5 个
  看跌指标: 3 个
  中性指标: 7 个
  总指标数: 15 个

=== 交易建议 ===
建议: 保持观望
理由: 多空信号相对均衡

信号可靠性: 高 (基于信号强度 0.03)
⚠️ 注意: 信号强度较低，建议结合其他分析方法


### 2.5 支撑阻力位分析

In [12]:
# 识别支撑阻力位
levels = analyzer.identify_support_resistance(window=15, min_touches=2)

print("=== 支撑阻力位分析 ===")
print()
print(f"当前价格: ${current_price:.2f}")
print()

# 支撑位
support_levels = levels['support_levels']
if support_levels:
    print(f"支撑位 ({len(support_levels)} 个):")
    # 找到最近的支撑位
    nearby_support = [level for level in support_levels if level < current_price]
    if nearby_support:
        nearest_support = max(nearby_support)
        distance = ((current_price - nearest_support) / current_price) * 100
        print(f"  最近支撑: ${nearest_support:.2f} (距离当前价格 {distance:.1f}%)")
    
    # 显示所有支撑位
    for i, level in enumerate(sorted(support_levels, reverse=True)[:5]):
        print(f"  S{i+1}: ${level:.2f}")
else:
    print("支撑位: 未发现明显支撑位")

print()

# 阻力位
resistance_levels = levels['resistance_levels']
if resistance_levels:
    print(f"阻力位 ({len(resistance_levels)} 个):")
    # 找到最近的阻力位
    nearby_resistance = [level for level in resistance_levels if level > current_price]
    if nearby_resistance:
        nearest_resistance = min(nearby_resistance)
        distance = ((nearest_resistance - current_price) / current_price) * 100
        print(f"  最近阻力: ${nearest_resistance:.2f} (距离当前价格 {distance:.1f}%)")
    
    # 显示所有阻力位
    for i, level in enumerate(sorted(resistance_levels)[:5]):
        print(f"  R{i+1}: ${level:.2f}")
else:
    print("阻力位: 未发现明显阻力位")

=== 支撑阻力位分析 ===

当前价格: $232.14

支撑位: 未发现明显支撑位

阻力位 (1 个):
  R1: $214.04


### 2.6 生成完整分析报告

In [13]:
# 生成分析报告
report = analyzer.generate_analysis_report()
print("=== 完整技术分析报告 ===")
print(report)

=== 完整技术分析报告 ===

技术分析报告

当前价格: 232.14

综合信号: 中性 (信号强度: 0.03)

信号分解:
• 趋势信号: -0.20
• 震荡信号: -0.17
• 成交量信号: 1.00

信号统计:
• 看涨指标: 5 个
• 看跌指标: 3 个
• 中性指标: 7 个
• 信号一致性: 0.64

支撑阻力位:
• 支撑位: 
• 阻力位: 

关键指标当前状态:
• RSI(60.4): 中性
• MACD(4.8499): 看跌
• KDJ(K:71.4, D:60.3): 中性
• 布林带位置(67.6%): 中性



## 3. 多股票对比分析

对比分析多只股票的技术指标

In [14]:
# 批量分析所有股票
analysis_results = {}

print("=== 多股票技术分析对比 ===")
print()

for symbol, data in stock_data.items():
    try:
        print(f"分析 {symbol}...")
        analyzer = TechnicalAnalyzer(data)
        analyzer.calculate_all_indicators()
        analyzer.generate_all_signals()
        
        signal, strength, analysis = analyzer.get_consensus_signal()
        
        analysis_results[symbol] = {
            'name': stocks[symbol],
            'current_price': data['close'].iloc[-1],
            'signal': signal,
            'strength': strength,
            'rsi': analyzer.indicators['rsi'].results['RSI'].iloc[-1],
            'macd': analyzer.indicators['macd'].results['MACD'].iloc[-1],
            'bb_position': analyzer.indicators['bb'].results['BB_Position'].iloc[-1],
            'analysis': analysis
        }
        
    except Exception as e:
        print(f"× {symbol} 分析失败: {e}")

print(f"\n完成! 成功分析 {len(analysis_results)} 只股票")

=== 多股票技术分析对比 ===

分析 AAPL...
计算技术指标...
技术指标计算完成
生成交易信号...
交易信号生成完成
分析 TSLA...
计算技术指标...
技术指标计算完成
生成交易信号...
交易信号生成完成
分析 GOOGL...
计算技术指标...
技术指标计算完成
生成交易信号...
交易信号生成完成
分析 MSFT...
计算技术指标...
技术指标计算完成
生成交易信号...
交易信号生成完成

完成! 成功分析 4 只股票


In [15]:
# 创建对比表格
comparison_data = []
signal_meanings = {2: "强烈看涨", 1: "看涨", 0: "中性", -1: "看跌", -2: "强烈看跌"}

for symbol, result in analysis_results.items():
    comparison_data.append({
        '股票代码': symbol,
        '公司名称': result['name'],
        '当前价格': f"${result['current_price']:.2f}",
        '综合信号': signal_meanings[result['signal']],
        '信号强度': f"{result['strength']:.2f}",
        'RSI': f"{result['rsi']:.1f}",
        'MACD': f"{result['macd']:.4f}",
        '布林带位置': f"{result['bb_position']:.1f}%",
        '看涨指标': result['analysis']['bullish_count'],
        '看跌指标': result['analysis']['bearish_count']
    })

# 转换为DataFrame并显示
comparison_df = pd.DataFrame(comparison_data)
print("股票技术分析对比表:")
print("=" * 120)
print(comparison_df.to_string(index=False))

# 按信号强度排序
print("\n\n按信号强度排序 (最强到最弱):")
print("=" * 60)
sorted_results = sorted(analysis_results.items(), key=lambda x: x[1]['strength'], reverse=True)
for i, (symbol, result) in enumerate(sorted_results, 1):
    print(f"{i}. {symbol} ({result['name'][:15]}...): {signal_meanings[result['signal']]} (强度: {result['strength']:.2f})")

股票技术分析对比表:
 股票代码            公司名称    当前价格 综合信号 信号强度  RSI    MACD 布林带位置  看涨指标  看跌指标
 AAPL      Apple Inc. $232.14   中性 0.03 60.4  4.8499 67.6%     5     3
 TSLA      Tesla Inc. $333.87   中性 0.20 46.8  5.9434 52.6%     5     2
GOOGL   Alphabet Inc. $212.91   中性 0.24 72.4  5.5244 99.3%     9     3
 MSFT Microsoft Corp. $506.69   中性 0.24 36.9 -0.9250 27.3%     1     7


按信号强度排序 (最强到最弱):
1. MSFT (Microsoft Corp....): 中性 (强度: 0.24)
2. GOOGL (Alphabet Inc....): 中性 (强度: 0.24)
3. TSLA (Tesla Inc....): 中性 (强度: 0.20)
4. AAPL (Apple Inc....): 中性 (强度: 0.03)


## 4. 技术指标可视化

绘制技术分析图表

In [ ]:
# 为AAPL绘制技术分析图表
if 'AAPL' in stock_data:
    print("生成 AAPL 技术分析图表...")
    
    # 重新创建分析器
    aapl_analyzer = TechnicalAnalyzer(stock_data['AAPL'])
    aapl_analyzer.calculate_all_indicators()
    
    # 调用可视化方法
    try:
        aapl_analyzer.plot_analysis(figsize=(15, 12))
        print("图表生成完成!")
    except Exception as e:
        print(f"图表生成失败: {e}")
        print("尝试生成简化版图表...")
        
        # 备选方案：简化版图表
        try:
            fig, ax = plt.subplots(2, 1, figsize=(12, 8))
            
            # 价格走势和移动平均线
            ax[0].plot(aapl_data['close'].values, label='Close Price', linewidth=2)
            if 'ma' in aapl_analyzer.indicators:
                ma_results = aapl_analyzer.indicators['ma'].results
                ax[0].plot(ma_results['SMA_20'].values, label='MA20', alpha=0.7)
                ax[0].plot(ma_results['SMA_50'].values, label='MA50', alpha=0.7)
            ax[0].set_title('AAPL Price & Moving Averages')
            ax[0].legend()
            ax[0].grid(True, alpha=0.3)
            
            # RSI
            if 'rsi' in aapl_analyzer.indicators:
                rsi_data = aapl_analyzer.indicators['rsi'].results['RSI']
                ax[1].plot(rsi_data.values, label='RSI', color='purple')
                ax[1].axhline(y=70, color='red', linestyle='--', alpha=0.5, label='Overbought')
                ax[1].axhline(y=30, color='green', linestyle='--', alpha=0.5, label='Oversold')
                ax[1].set_title('RSI Indicator')
                ax[1].set_ylim(0, 100)
                ax[1].legend()
                ax[1].grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.show()
            print("简化版图表生成完成!")
        except Exception as e2:
            print(f"简化版图表也生成失败: {e2}")
            print("可能是matplotlib配置问题，请检查字体设置")
else:
    print("未找到AAPL数据")

In [ ]:
# 备用图表方案：基础技术指标可视化
def create_basic_charts(data, analyzer, symbol):
    """创建基础的技术分析图表（无中文标签）"""
    try:
        # 创建子图
        fig = plt.figure(figsize=(15, 10))
        
        # 1. 价格和移动平均线
        ax1 = plt.subplot(3, 1, 1)
        plt.plot(data['close'], label=f'{symbol} Close', linewidth=2, color='blue')
        
        if 'ma' in analyzer.indicators:
            ma_results = analyzer.indicators['ma'].results
            if 'SMA_20' in ma_results:
                plt.plot(ma_results['SMA_20'], label='MA20', alpha=0.8, color='orange')
            if 'SMA_50' in ma_results:
                plt.plot(ma_results['SMA_50'], label='MA50', alpha=0.8, color='green')
        
        plt.title(f'{symbol} Price Chart with Moving Averages')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 2. MACD
        ax2 = plt.subplot(3, 1, 2)
        if 'macd' in analyzer.indicators:
            macd_results = analyzer.indicators['macd'].results
            plt.plot(macd_results['MACD'], label='MACD', color='blue')
            plt.plot(macd_results['Signal'], label='Signal', color='red')
            plt.bar(range(len(macd_results['Histogram'])), macd_results['Histogram'], 
                   label='Histogram', alpha=0.6, color='gray')
            plt.axhline(y=0, color='black', linestyle='-', alpha=0.5)
        
        plt.title('MACD Indicator')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 3. RSI
        ax3 = plt.subplot(3, 1, 3)
        if 'rsi' in analyzer.indicators:
            rsi_data = analyzer.indicators['rsi'].results['RSI']
            plt.plot(rsi_data, label='RSI', color='purple', linewidth=2)
            plt.axhline(y=70, color='red', linestyle='--', alpha=0.7, label='Overbought (70)')
            plt.axhline(y=30, color='green', linestyle='--', alpha=0.7, label='Oversold (30)')
            plt.ylim(0, 100)
        
        plt.title('RSI Indicator')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.xlabel('Time Period')
        
        plt.tight_layout()
        plt.show()
        return True
        
    except Exception as e:
        print(f\"基础图表生成失败: {e}\")\n        return False

# 测试基础图表功能
if 'AAPL' in stock_data:
    print(\"尝试生成基础技术分析图表...\")\n    success = create_basic_charts(stock_data['AAPL'], aapl_analyzer, 'AAPL')
    if success:
        print(\"基础图表生成成功！\")\nelse:\n    print(\"无AAPL数据可用\")

## 5. 投资组合建议

基于技术分析结果给出投资组合建议

In [18]:
print("=== 基于技术分析的投资组合建议 ===")
print()

# 分类股票
strong_buy = []  # 强烈看涨
buy = []         # 看涨
hold = []        # 中性
sell = []        # 看跌
strong_sell = [] # 强烈看跌

for symbol, result in analysis_results.items():
    signal = result['signal']
    strength = result['strength']
    
    stock_info = {
        'symbol': symbol,
        'name': result['name'],
        'strength': strength,
        'price': result['current_price']
    }
    
    if signal == 2:
        strong_buy.append(stock_info)
    elif signal == 1:
        buy.append(stock_info)
    elif signal == 0:
        hold.append(stock_info)
    elif signal == -1:
        sell.append(stock_info)
    else:
        strong_sell.append(stock_info)

# 输出建议
def print_category(stocks, title, recommendation):
    if stocks:
        print(f"{title} ({len(stocks)} 只):")
        for stock in sorted(stocks, key=lambda x: x['strength'], reverse=True):
            print(f"  • {stock['symbol']} - {stock['name'][:20]}... (强度: {stock['strength']:.2f})")
        print(f"  建议: {recommendation}")
        print()

print_category(strong_buy, "🚀 强烈看涨", "重点关注，可考虑重仓")
print_category(buy, "📈 看涨", "适量买入，建议配置")
print_category(hold, "➡️ 中性", "持有观望，等待信号明确")
print_category(sell, "📉 看跌", "考虑减仓或卖出")
print_category(strong_sell, "💥 强烈看跌", "建议立即卖出")

# 投资组合权重建议
total_positive = len(strong_buy) + len(buy)
if total_positive > 0:
    print("投资组合权重建议 (基于信号强度):")
    print("=" * 40)
    
    all_positive = strong_buy + buy
    total_strength = sum(stock['strength'] for stock in all_positive)
    
    for stock in sorted(all_positive, key=lambda x: x['strength'], reverse=True):
        weight = (stock['strength'] / total_strength) * 100 if total_strength > 0 else 0
        print(f"  {stock['symbol']}: {weight:.1f}% (强度: {stock['strength']:.2f})")
else:
    print("当前技术面偏弱，建议以现金为主，等待更好的买入时机")

=== 基于技术分析的投资组合建议 ===

➡️ 中性 (4 只):
  • MSFT - Microsoft Corp.... (强度: 0.24)
  • GOOGL - Alphabet Inc.... (强度: 0.24)
  • TSLA - Tesla Inc.... (强度: 0.20)
  • AAPL - Apple Inc.... (强度: 0.03)
  建议: 持有观望，等待信号明确

当前技术面偏弱，建议以现金为主，等待更好的买入时机


## 6. 总结与风险提示

技术分析结果总结

In [19]:
print("=== 技术分析总结 ===")
print()

# 统计信息
total_stocks = len(analysis_results)
avg_strength = np.mean([result['strength'] for result in analysis_results.values()])
bullish_count = sum(1 for result in analysis_results.values() if result['signal'] > 0)
bearish_count = sum(1 for result in analysis_results.values() if result['signal'] < 0)
neutral_count = sum(1 for result in analysis_results.values() if result['signal'] == 0)

print(f"分析股票总数: {total_stocks}")
print(f"平均信号强度: {avg_strength:.2f}")
print(f"看涨股票: {bullish_count} ({bullish_count/total_stocks*100:.1f}%)")
print(f"看跌股票: {bearish_count} ({bearish_count/total_stocks*100:.1f}%)")
print(f"中性股票: {neutral_count} ({neutral_count/total_stocks*100:.1f}%)")
print()

# 市场情绪判断
if bullish_count > bearish_count:
    market_sentiment = "偏乐观"
    sentiment_desc = "大多数技术指标显示积极信号"
elif bearish_count > bullish_count:
    market_sentiment = "偏悲观"
    sentiment_desc = "大多数技术指标显示消极信号"
else:
    market_sentiment = "中性"
    sentiment_desc = "多空信号相对均衡"

print(f"当前市场技术面: {market_sentiment}")
print(f"解读: {sentiment_desc}")

print("\n=== ⚠️ 重要风险提示 ===")
print()
print("1. 技术分析的局限性:")
print("   • 技术指标基于历史数据，存在滞后性")
print("   • 震荡市场中容易产生假突破信号")
print("   • 不能预测重大突发事件的影响")
print()
print("2. 投资建议:")
print("   • 技术分析应与基本面分析结合使用")
print("   • 不要依赖单一指标做投资决策")
print("   • 严格执行止损策略，控制风险")
print("   • 合理配置仓位，避免重仓单一股票")
print()
print("3. 免责声明:")
print("   • 本分析仅供参考，不构成投资建议")
print("   • 投资有风险，决策需谨慎")
print("   • 请根据自身情况做出投资决定")

print("\n=== 分析完成 ===")
print(f"生成时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("感谢使用 quantlib 技术分析模块！")

=== 技术分析总结 ===

分析股票总数: 4
平均信号强度: 0.18
看涨股票: 0 (0.0%)
看跌股票: 0 (0.0%)
中性股票: 4 (100.0%)

当前市场技术面: 中性
解读: 多空信号相对均衡

=== ⚠️ 重要风险提示 ===

1. 技术分析的局限性:
   • 技术指标基于历史数据，存在滞后性
   • 震荡市场中容易产生假突破信号
   • 不能预测重大突发事件的影响

2. 投资建议:
   • 技术分析应与基本面分析结合使用
   • 不要依赖单一指标做投资决策
   • 严格执行止损策略，控制风险
   • 合理配置仓位，避免重仓单一股票

3. 免责声明:
   • 本分析仅供参考，不构成投资建议
   • 投资有风险，决策需谨慎
   • 请根据自身情况做出投资决定

=== 分析完成 ===
生成时间: 2025-08-31 20:58:15
感谢使用 quantlib 技术分析模块！
